In [2]:
from werkzeug.wrappers import Request, Response
from jinja2 import Template
from datetime import datetime, timedelta
import collections
from datetime import datetime

from amigocloud import AmigoCloud

In [12]:
import pandas as pd

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [8]:
query_new = {'query' : 'select id, canhero from dataset_292618 where canhero is not null and (link_informe is null or link_informe=\'\')'}
project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
select_new = amigocloud.get(project, query_new)

In [10]:
select_new['data']

[{'id': 110, 'canhero': '40256 / INSERVA DAMIANA S.R.L.'},
 {'id': 89, 'canhero': '7506 / LOHENER ROSENDA SHEILA PAZ DE'},
 {'id': 90, 'canhero': '5011 / FRANCO ARIAS GREGORIO'}]

In [13]:
df_select = pd.DataFrame(select_new['data'])

In [14]:
df_select

,id,canhero
0,110,40256 / INSERVA DAMIANA S.R.L.
1,89,7506 / LOHENER ROSENDA SHEILA PAZ DE
2,90,5011 / FRANCO ARIAS GREGORIO


In [29]:
for i, r in df_select.iterrows():
    print(r[0])

110
89
90


In [ ]:
def generar_reporte(cod):
    id = cod
    # DATOS DE LAS INPECCION
    #obtener datos de inspeccion
    query = {'query': 'select CURRENT_DATE, * from dataset_292618 where id = {id}'.format(id=id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    #cambio de formato de la fecha
    data['date'] = datetime.strptime(data['date'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    #conversion a coleecion
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # DATOS DE LA PROPIEDAD
    query = {'query': 'select * from dataset_293179 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
    select = amigocloud.get(project, query)
    propiedades_data = select['data']
    
    #crear lista gral de propiedades
    propiedades = []
    #recorrer todos los lotes propiedad por propiedad
    for prop in propiedades_data:
        #crea campo 'lote' en la propiedad seleccionada
        prop['lote'] = []
        #seleccionar lotes de la propiedad
        query_lotes = {'query': 'select * from dataset_293180 where prop_ref_lote = \'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
        project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
        lotes_select = amigocloud.get(project, query_lotes)
        #recorrer los lotes
        for lote_select in lotes_select['data']:
            #convertir el lote en objeto
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            #agragar obj lote a las lista de lotes de la propiedad
            prop['lote'].append(lote)
        #agregar propiedad a lista general de propiedades
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
    
    # FOTOS
    #lista de fotos
    fotos = []
    #recorrer todas las propiedades
    for propiedad in propiedades:
        #recorrer todos los lotos
        for lote in propiedad.lote:
            print(lote.lote, lote.amigo_id)
            #consultar fotos del lote
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_42303 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
            fotos_filename = amigocloud.get(project, query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)
    fotos

In [30]:
id=112

In [ ]:
# DATOS DE LAS INPECCION
query = {'query': 'select CURRENT_DATE, * from dataset_292618 where id = {id}'.format(id=id)}
project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
data['date'] = datetime.strptime(data['date'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

end


In [7]:
insp

insp(date='18/09/2023', fecha_registro='2023-09-15 14:19:40+00:00', amigo_id='c08fe291239b4e3cac1e7ebc0ed193e5', fecha_quema='15/09/2023', fecha_inspeccion='15/09/2023', fecha_update='2023-09-15 14:19:40+00:00', canhero='13132 / ZUMARAN MENACHO JORGE', superficie_total=59.56, rendimiento=60.0, produccion=3573.6, responsable='Rogelio Acuña Rodríguez', inicio_incendio='Fuera de la propiedad', causa='Origen desconocido', responsable_de_quema='CAZADOR', observaciones=None, link_informe=None, link_informe_editable=None, cite=362, link_comprobante=None, id=112, tipo_cosecha='SEMI-MECANIZADO')

In [8]:
# DATOS DE LA PROPIEDAD
query = {'query': 'select * from dataset_293179 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
select = amigocloud.get(project, query)
propiedades_data = select['data']
print('end')

end


In [9]:
propiedades_data

[{'ubicacion': '0101000020E610000060CF93E081B84FC070A0B1C66B5F31C0',
  'amigo_id': '57c2b2cc4eef4e5fae5a3b7b0f15a4c0',
  'fecha_registro': '2023-09-15 14:07:04+00:00',
  'fecha_update': '2023-09-15 14:22:00.654234+00:00',
  'propiedad': '1719 / SANTA FE--ZUMARAN JORGE',
  'tenencia': 'Alquilado',
  'insp_ref_prop': 'c08fe291239b4e3cac1e7ebc0ed193e5'}]

In [10]:
propiedades = []
for prop in propiedades_data:
    prop['lote'] = []
    query_lotes = {'query': 'select * from dataset_293180 where prop_ref_lote = \'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
    project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
    lotes_select = amigocloud.get(project, query_lotes)
    for lote_select in lotes_select['data']:
        lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
        prop['lote'].append(lote)
    propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
print('end')

end


In [12]:
# fotos lotes
fotos = []
for propiedad in propiedades:
    for lote in propiedad.lote:
        print(lote.lote, lote.amigo_id)
        query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_42303 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
        project = 'https://app.amigocloud.com/api/v1/projects/31874/sql'
        fotos_filename = amigocloud.get(project, query_fotos)
        if fotos_filename['data'] != []:
            dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
            fotos.append(dic)
fotos

L2 d6fce63f2449417f8363d8ccc51e8b0c
L3 6fca2c8ceef941e482957bdd5203d9f4
L4 9e59a1a1c05d4fcd99fd6fb1e16357ac
L5 a47f97db33234ba085f443ec1ad885c8
L6 422447e4bf90467cbc157c98c5be1303
L1 c410da6eb01f459b9a67022464bd111b


[{'lote': 'L2', 'file_name': 'WhatsApp Image 2023-09-15 at 9.21.41 AM.jpeg'},
 {'lote': 'L1', 'file_name': 'WhatsApp Image 2023-09-15 at 9.21.40 AM.jpeg'}]

In [26]:
from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

In [27]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [28]:
i = 0
lista_planos = []
path = ''
for propiedad in propiedades:
    i = i + 1
    lotes_lista = []
    for lote in propiedad.lote:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    path = ruta + '/planos/' + str(insp.amigo_id) + '_' + str(propiedad.amigo_id) + '.jpeg'
    lista_planos.append(path)
    fig.savefig(path, dpi = 300, bbox_inches='tight')
    del fig
print('end')

NameError: name 'propiedades' is not defined

In [16]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate(ruta + "/templates/tpl_infome_quema.docx")

#generar lista de InlineImage de planos 
lista_InlineImage = []
for plano in lista_planos:
    lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    file = open(ruta + '/fotos/' + foto['file_name'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})

firma_respon = None
if insp.responsable == 'Rogelio Acuña Rodríguez':
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_rogelio.png', width=Mm(60))
else:
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_jaldin.png', width=Mm(60))

context = {'insp':insp, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.canhero.split(' / ')
file_name = cod_nom[0] + '_CAQ_' + insp.fecha_inspeccion.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

doc.save(ruta + '/informes/_' + file_name + '.docx')